In [39]:
import os
import shutil
from PIL import Image

In [40]:
data_dir = '../data'
dataset_dir = os.path.join(data_dir, 'DATASET')

print('data_dir:', dataset_dir)
print('os.listdir(dataset_dir):', os.listdir(dataset_dir))

data_dir: ../data/DATASET
os.listdir(dataset_dir): ['TRAIN', 'TEST']


In [41]:


new_images_dir = os.path.join(data_dir, 'images')

if os.path.exists(new_images_dir):
    shutil.rmtree(new_images_dir)
os.makedirs(new_images_dir)

In [42]:
def create_dir_if_not_exists(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

In [43]:
targets_images = {} # example: {'godness': ['../data/images/godness/1.jpg', '../data/DATASET/godness/2.jpg', ...], ...}


# enter in all the subdirectories of the dataset directory
for sub_dir in os.listdir(dataset_dir):
    print('sub_dir:', sub_dir)
    sub_dir_path = os.path.join(dataset_dir, sub_dir)

    # enter in all subdirectories of the sub_dir
    for sub_sub_dir in os.listdir(sub_dir_path):
        print('|_sub_sub_dir:', sub_sub_dir)
        
        if sub_sub_dir not in targets_images:
            targets_images[sub_sub_dir] = []
        
        sub_sub_dir_path = os.path.join(sub_dir_path, sub_sub_dir)
        new_sub_sub_dir = os.path.join(new_images_dir, sub_sub_dir)
        create_dir_if_not_exists(new_sub_sub_dir)
        
        # enter in all the files of the sub_sub_dir
        for file in os.listdir(sub_sub_dir_path):
            file_path = os.path.join(sub_sub_dir_path, file)
            new_file_path = os.path.join(new_sub_sub_dir, file)
            if os.path.exists(new_file_path):
                print('file already exists:', new_file_path)
                continue
            shutil.copy(file_path, new_file_path)
            targets_images[sub_sub_dir].append(new_file_path)

sub_dir: TRAIN
|_sub_sub_dir: goddess
|_sub_sub_dir: tree
|_sub_sub_dir: downdog
|_sub_sub_dir: plank
|_sub_sub_dir: warrior2
sub_dir: TEST
|_sub_sub_dir: goddess
|_sub_sub_dir: tree
|_sub_sub_dir: downdog
|_sub_sub_dir: plank
|_sub_sub_dir: warrior2


In [44]:
targets_images.keys()
count = 0
for key in targets_images.keys():
    print('key:', key)
    print('len(targets_images[key]):', len(targets_images[key]))
    print('')
    count += len(targets_images[key])

print('count:', count)

def get_extension(file_path):
    return file_path.split('.')[-1]

def get_extensions(target):
    extensions = {}
    for target in targets_images.keys():
        for image_path in targets_images[target]:
            extension = get_extension(image_path)
            if extension not in extensions:
                extensions[extension] = 0
            extensions[extension] += 1
    return extensions

key: goddess
len(targets_images[key]): 260

key: tree
len(targets_images[key]): 229

key: downdog
len(targets_images[key]): 320

key: plank
len(targets_images[key]): 381

key: warrior2
len(targets_images[key]): 361

count: 1551


In [45]:
get_extensions(targets_images)

{'jpg': 1410, 'png': 110, 'JPG': 19, 'jpeg': 10, 'bmp': 1, 'PNG': 1}

In [46]:
# renaming all extensions to lowercase
for target in targets_images.keys():
    for index, image_path in enumerate(targets_images[target]):
        extension = get_extension(image_path)
        lower_extension = extension.lower()
        new_image_path = image_path.replace(extension,lower_extension)
        os.rename(image_path,new_image_path)
        targets_images[target][index] = new_image_path



In [47]:
get_extensions(targets_images)

{'jpg': 1429, 'png': 111, 'jpeg': 10, 'bmp': 1}

In [48]:
# convert all images to jpg
for target in targets_images.keys():
    for index, image_path in enumerate(targets_images[target]):
        extension = get_extension(image_path)
        if extension == 'jpg':
            continue
        new_image_path = image_path.replace(extension, 'jpg')
        im = Image.open(image_path)
        rgb_im = im.convert("RGB")
        rgb_im.save(new_image_path)
        os.remove(image_path)
        targets_images[target][index] = new_image_path
        


/usr/lib/python3/dist-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [50]:
get_extensions(targets_images)

{'jpg': 1551}

In [53]:
import numpy as np
import matplotlib.pyplot as plt

In [54]:
np.random.seed(20)

In [ ]:
def plot_images(datagen):
  # generate samples and plot
  for i in range(9):
    # define subplot
    plt.subplot(330 + 1 + i)
    # generate batch of images
    batch = datagen.next()
    # convert to unsigned integers for viewing
    image_ = batch[0]
    # plot raw pixel data
    plt.imshow(image_[0])
  plt.show()
